In [70]:
from dgl.data import citation_graph,CoraDataset,TUDataset,GINDataset, ICEWS18
import torch
from torch import nn
from dgl import DGLGraph
import numpy as np
import dgl
from datasets.prepocessing import one_class_processing

In [3]:
torch.cuda.is_available()

False

In [ ]:
a=torch.ones(100,32)
b=torch.ones(100,32)

In [ ]:
data =citation_graph.load_cora()
#data =citation_graph.load_citeseer()
#data =TUDataset('PROTEINS_full')
#data =ICEWS18('train')

In [179]:
def pre_process(args,dataset):

    for i in range(len(dataset)):
        print(i)
        #dataset.graph_lists[i].ndata
        normal_idx=torch.where(dataset.graph_lists[i].ndata['node_labels']==0)[0]
        abnormal_idx=torch.where(dataset.graph_lists[i].ndata['node_labels']!=0)[0]
        dataset.graph_lists[i].ndata['node_labels'][normal_idx]=0
        dataset.graph_lists[i].ndata['node_labels'][abnormal_idx]=1
        
        g=dgl.transform.add_self_loop(dataset.graph_lists[i])
        g.ndata.update(dataset.graph_lists[i].ndata)
        dataset.graph_lists[i]=g
        print(dataset.graph_lists[i].ndata['node_labels'].max())
        #print(dataset.graph_lists[i])
    return dataset

In [180]:
from dgl.data import tu
dataset = tu.TUDataset(name='ENZYMES')
print(len(dataset))
train_size = int(0.6 * len(dataset))
test_size = int(0.2 * len(dataset))
val_size = int(len(dataset) - train_size - test_size)
dataset = pre_process(None,dataset)
dataset_train, dataset_val, dataset_test = torch.utils.data.random_split(
    dataset, (train_size, val_size, test_size))

600
0
tensor(1)


In [178]:
for i in range(len(dataset)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [150]:
c=torch.ones(32)
x=torch.ones(50,32)+10
print(x)
print(c)

tensor([[11., 11., 11.,  ..., 11., 11., 11.],
        [11., 11., 11.,  ..., 11., 11., 11.],
        [11., 11., 11.,  ..., 11., 11., 11.],
        ...,
        [11., 11., 11.,  ..., 11., 11., 11.],
        [11., 11., 11.,  ..., 11., 11., 11.],
        [11., 11., 11.,  ..., 11., 11., 11.]])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [151]:
t=(x - c)** 2
t.size()

torch.Size([50, 32])

In [152]:
dist1 = torch.sum((x - c) ** 2, dim=1)
print(dist1.size())
dist1

torch.Size([50])


tensor([3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200.,
        3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200.,
        3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200.,
        3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200.,
        3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200., 3200.])

In [153]:
torch.mean(dist1)

tensor(3200.)

In [154]:
c_=c.repeat(x.size()[0],1)
# # print('c',c.size())
res=x-c_
# # print('res',res.size())
res=torch.mean(res, 1, keepdim=True)
# # print('res mean',res.size())
dist2=torch.diag(torch.mm(res,torch.transpose(res, 0, 1)))
print(dist2.size())
dist2

torch.Size([50])


tensor([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
        100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
        100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
        100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
        100., 100.])

In [155]:
torch.mean(dist2)

tensor(100.)

In [36]:
torch.where(dataset.graph_lists[0].ndata['node_labels']==0)[0]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21])

In [10]:
def batching_graph(batch):
    '''
    for dataset batching
    transform ndata to tensor (in gpu is available)
    '''
    graphs, labels = map(list, zip(*batch))
    #cuda = torch.cuda.is_available()

    # batch graphs and cast to PyTorch tensor
    for graph in graphs:
        for (key, value) in graph.ndata.items():
            graph.ndata[key] = value.float()
    batched_graphs = dgl.batch(graphs)

    # cast to PyTorch tensor
    batched_labels = torch.LongTensor(np.array(labels))

    return batched_graphs, batched_labels

In [43]:
train_loader=torch.utils.data.DataLoader(dataset_train,
                                       batch_size=32,
                                       shuffle=True,
                                       collate_fn=batching_graph,
                                       drop_last=True,
                                       num_workers=4)

In [156]:
for (batch_idx, (batch_graph, graph_labels)) in enumerate(train_loader):
    print(batch_idx)
    if batch_idx >3:
        break

0
1
2
3
4


In [74]:
a=batch_graph.ndata['node_attr']
m = nn.BatchNorm1d(29, affine=False)

In [79]:
batch_graph.ndata['node_attr'].shape[1]

29

In [172]:
a=np.ones(10)
a

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [174]:
b=np.append(a,None)
b

array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, None],
      dtype=object)

In [76]:
output.max()

tensor(16.6918)

In [34]:
normal_idx=torch.where(batch_graph.ndata['node_labels']==0)[0]
abnormal_idx=torch.where(batch_graph.ndata['node_labels']!=0)[0]


normal_idx

tensor([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
          24,   25,   38,   39,   40,   41,   42,   43,   44,   45,   46,   47,
          48,   49,   50,   51,   52,   53,   54,   55,   56,   57,   71,   78,
          79,   80,   81,   82,   83,   84,   85,   86,   87,   88,   89,   90,
          91,   92,  135,  136,  137,  138,  139,  140,  141,  142,  143,  144,
         145,  175,  176,  177,  178,  179,  180,  181,  182,  183,  184,  185,
         186,  187,  197,  198,  199,  200,  201,  202,  203,  204,  205,  206,
         207,  208,  209,  210,  211,  212,  213,  214,  215,  216,  217,  218,
         219,  220,  221,  222,  223,  224,  225,  226,  227,  228,  229,  230,
         261,  262,  263,  264,  265,  266,  271,  272,  273,  274,  305,  306,
         307,  308,  309,  310,  311,  312,  313,  314,  315,  332,  333,  334,
         335,  336,  343,  344,  345,  346,  347,  348,  349,  350,  351,  352,
         353,  354,  355,  356,  357,  3

In [29]:
batch_graph.ndata['node_labels'][normal_idx]=0
batch_graph.ndata['node_labels'][abnormal_idx]=1

In [32]:
batch_graph.ndata['node_labels'].sum()

tensor(593.)

In [ ]:
input_dim, label_dim, max_num_node = dataset.statistics()

In [ ]:
print(len(dataset_train))
print(len(dataset_val))
print(len(dataset_test))

In [ ]:
label0=data[0][0].ndata['node_labels']
label1=data[1][0].ndata['node_labels']
label2=data[2][0].ndata['node_labels']

In [ ]:
label1

In [ ]:
print(len(label0))
print(len(label1))
print(len(label2))

In [ ]:
data[1][0].ndata['node_labels'].size()

In [ ]:
labels,train_mask,val_mask,test_mask=one_class_processing(data[0][0],4)

In [ ]:
from collections import Counter
Counter(data.labels)

In [ ]:
normal_class=2
labels,train_mask,val_mask,test_mask=one_class_processing(data.labels,normal_class)

In [ ]:
labels[train_mask.astype(int)].shape

In [ ]:
print('train',np.sum(labels[train_mask.astype(int)==1]))
print('val',np.sum(labels[val_mask.astype(int)==1]))
print('test',np.sum(labels[test_mask.astype(int)==1]))

In [ ]:
print('train',np.sum(train_mask))
print('val',np.sum(val_mask))
print('test',np.sum(test_mask))

In [ ]:
train_mask.shape